# Import các thư viện cần thiết 

In [1]:
import numpy as np
import pandas as pd

In [2]:
from collections import namedtuple
from itertools import combinations
from itertools import chain

# Đọc dữ liệu vào dataframe từ file csv

In [3]:
df=pd.read_csv(r"data.csv",header=None)

# Chuyển dataframe thành dạng list

In [4]:
records=[]
for i in range(0,df.shape[0]):
    records.append([str(df.values[i,j]) for j  in range(0,6)])

# Định nghĩa một bản ghi (record) SupportRecord bao gồm các trường items và support

namedtuple cũng giống như tuple bình thường nhưng không những cho phép ta truy cập theo chỉ số (index) mà còn theo tên thuộc tính

In [5]:
SupportRecord = namedtuple(
    'SupportRecord', ('items', 'support'))  # định nghĩa một bản ghi (record) SupportRecord bao gồm các trường items và support

# Hàm tính số lượng giao dịch, số lượng item và chỉ số index của chúng trong các giao dịch

In [6]:
def calc_num_items_index(transactions):
    num_transaction = 0  # Tổng số giao dịch
    items = []  # Danh sách các item
    items_index = {}  # Chỉ số index của item trong danh sách giao dịch
    for transaction in transactions:
        for item in transaction:
            if item == 'nan': # Bỏ qua các giá trị nan
                continue
            if item not in items_index:
                items.append(item)
                items_index[item] = set()
            items_index[item].add(num_transaction)
        num_transaction += 1
    return num_transaction, items, items_index


# Hàm để tạo ra các ứng viên tiếp theo cho thuật toán Vertical Apriori

In [7]:
def create_next_candidates(prev_candidates, length):
    # Tạo danh sách các mặt hàng đã được sắp xếp
    # fronzenset cũng giống như set nhưng nó immutable nên có thể sử dụng như là key của dictionary
    items = sorted(frozenset(chain.from_iterable(prev_candidates)))
    # Tạo danh sách các ứng viên tiếp theo bằng cách lấy tất cả các kết hợp có độ dài là "length"
    tmp_next_candidates = (frozenset(x) for x in combinations(items, length))
    # Nếu độ dài "length" nhỏ hơn 3 thì trả về toàn bộ ứng viên vì các tập con của nó cũng giống như tập items
    if length < 3:
        return list(tmp_next_candidates)
    # Tạo danh sách các ứng viên tiếp theo chỉ bao gồm các ứng viên có kích thước là "length" và tất cả các tập con kích thước "length - 1" thuộc danh sách các ứng viên trước đó
    next_candidates = [
        candidate for candidate in tmp_next_candidates
        if all(
            frozenset(x) in prev_candidates
            for x in combinations(candidate, length - 1))
    ]
    # Trả về danh sách các ứng viên tiếp theo
    return next_candidates


# Hàm Vertical Apriori

Ở trong file pdf, support của một itemset là số lượng giao dịch chứa itemset, tuy nhiên để giống với định nghĩa support, cũng như để dễ so sánh với hàm ECLAT, support ở đây có ý nghĩa là số lượng giao dịch chứa itemset chia cho tổng số lượng giao dịch

Ví dụ min_support = 3 trong file pdf tương ứng với min_support = 3/22 ( vì có tổng 22 giao dịch)

In [8]:

def vertical_apriori(transactions, min_support, min_combination=1, max_combination=None):
    # tính toán các chỉ số cho các giao dịch, danh sách các mục trong các giao dịch và một bản đồ từ mục đến các giao dịch chứa chúng
    # num_transaction là số giao dịch trong transactions.
    # transaction_items là danh sách các item trong các giao dịch.
    # transaction_index là danh sách index của các item  trong các giao dịch chứa chúng
    num_transaction, transaction_items, transaction_index = calc_num_items_index(
        transactions)
    # Nếu max_combination không được chỉ định, nó sẽ bao gồm tất cả các item trong các giao dịch
    if max_combination is None:
        max_combination = len(transaction_items)
    # Khởi tạo tập ứng viên đầu tiên chỉ chứa các item đơn lẻ.
    candidates = [frozenset([item]) for item in transaction_items]
    length = 1
    support_records = []  # Danh sách chứa các SupportRecord thỏa mãn điều kiện
    # Tiếp tục tạo các tập ứng viên mới cho đến khi không còn ứng viên nào thỏa mãn điều kiện.
    while candidates:
        # Tập hỗ trợ (support set) chứa các tập ứng viên thỏa mãn điều kiện.
        relations = set()
        for relation_candidate in candidates:
            # Tính support của tập ứng viên hiện tại.
            support = len(set.intersection(
                *[transaction_index[item] for item in relation_candidate]))
            if support < min_support*num_transaction:
                continue  # Nếu support nhỏ hơn min_support, bỏ qua.
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
            
            # Chỉ lấy các support record có số lượng item >= min_combination
            if len(candidate_set) < min_combination:
                continue
            support_records.append(SupportRecord(
                candidate_set, support/num_transaction))
        length += 1
        # Chỉ lấy các support record có số lượng items <= max_combination
        if length > max_combination:
            break
        # Tạo các ứng viên mới có độ dài tăng dần.
        candidates = create_next_candidates(relations, length)
    # Trả về danh sách các SupportRecord thỏa mãn điều kiện.
    return support_records


# Xử lý kết quả thô để giống với kết quả của hàm ECLAT(không cần thiết vì nhìn kết quả thô cũng so sánh được )

In [9]:
output = vertical_apriori(records,3/22) # trường hợp min_support = 3/22 (tương đương với 3 trong file pdf)
res_dict={}
for res in  output:
    res_dict['&'.join(res.items)]=res.support
res_dict

{'Wine': 0.7272727272727273,
 'Chips': 0.6363636363636364,
 'Bread': 0.7272727272727273,
 'Butter': 0.6818181818181818,
 'Milk': 0.7727272727272727,
 'Apple': 0.6818181818181818,
 'Bread&Apple': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Chips': 0.45454545454545453,
 'Milk&Apple': 0.5,
 'Wine&Apple': 0.5,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Chips': 0.4090909090909091,
 'Milk&Bread': 0.5909090909090909,
 'Bread&Wine': 0.5909090909090909,
 'Butter&Chips': 0.4090909090909091,
 'Milk&Butter': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Milk&Chips': 0.45454545454545453,
 'Wine&Chips': 0.4090909090909091,
 'Milk&Wine': 0.6363636363636364,
 'Bread&Apple&Butter': 0.4090909090909091,
 'Bread&Apple&Chips': 0.36363636363636365,
 'Milk&Bread&Apple': 0.4090909090909091,
 'Bread&Apple&Wine': 0.45454545454545453,
 'Apple&Butter&Chips': 0.36363636363636365,
 'Milk&Apple&Butter': 0.4090909090909091,
 'Wine&Apple&Butter': 0.36363636363636365,
 'Milk&Apple&Chips': 0.3181818181818182,
 'Win

# So sánh với kết quả hàm ECLAT

In [10]:
from pyECLAT import ECLAT

In [11]:
eclat_instance=ECLAT(data=df,verbose=True)
indexes,supports = eclat_instance.fit(min_support=3/22,min_combination=1,max_combination=6,separator='&',verbose=True)
supports

100%|██████████| 6/6 [00:00<00:00, 1002.34it/s]


Combination 1 by 1


6it [00:00, 130.79it/s]


Combination 2 by 2


15it [00:00, 99.60it/s] 


...

Combination 5 by 5


6it [00:00, 65.39it/s]


Combination 6 by 6


1it [00:00, 40.10it/s]


{'Apple': 0.6818181818181818,
 'Chips': 0.6363636363636364,
 'Wine': 0.7272727272727273,
 'Bread': 0.7272727272727273,
 'Butter': 0.6818181818181818,
 'Milk': 0.7727272727272727,
 'Apple&Chips': 0.45454545454545453,
 'Apple&Wine': 0.5,
 'Apple&Bread': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Milk': 0.5,
 'Chips&Wine': 0.4090909090909091,
 'Chips&Bread': 0.4090909090909091,
 'Chips&Butter': 0.4090909090909091,
 'Chips&Milk': 0.45454545454545453,
 'Wine&Bread': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Wine&Milk': 0.6363636363636364,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Milk': 0.5909090909090909,
 'Butter&Milk': 0.5909090909090909,
 'Apple&Chips&Wine': 0.2727272727272727,
 'Apple&Chips&Bread': 0.36363636363636365,
 'Apple&Chips&Butter': 0.36363636363636365,
 'Apple&Chips&Milk': 0.3181818181818182,
 'Apple&Wine&Bread': 0.45454545454545453,
 'Apple&Wine&Butter': 0.36363636363636365,
 'Apple&Wine&Milk': 0.4090909090909091,
 'Apple&Bread&Butter': 0.4090909090909091,
 'Apple

# So sánh  2 kết quả thấy giống nhau 

# Một số bộ test khác để so sánh 2 kết quả

## Test 1

min_support = 0.4, min_combination = 2, max_combination =5

In [12]:
output = vertical_apriori(records,0.4, min_combination=2, max_combination=5)
res_dict={}
for res in  output:
    res_dict['&'.join(res.items)]=res.support
res_dict

{'Bread&Apple': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Chips': 0.45454545454545453,
 'Milk&Apple': 0.5,
 'Wine&Apple': 0.5,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Chips': 0.4090909090909091,
 'Milk&Bread': 0.5909090909090909,
 'Bread&Wine': 0.5909090909090909,
 'Butter&Chips': 0.4090909090909091,
 'Milk&Butter': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Milk&Chips': 0.45454545454545453,
 'Wine&Chips': 0.4090909090909091,
 'Milk&Wine': 0.6363636363636364,
 'Bread&Apple&Butter': 0.4090909090909091,
 'Milk&Bread&Apple': 0.4090909090909091,
 'Bread&Apple&Wine': 0.45454545454545453,
 'Milk&Apple&Butter': 0.4090909090909091,
 'Milk&Apple&Wine': 0.4090909090909091,
 'Milk&Bread&Butter': 0.5,
 'Bread&Butter&Wine': 0.45454545454545453,
 'Milk&Bread&Wine': 0.5,
 'Milk&Butter&Wine': 0.45454545454545453,
 'Milk&Bread&Butter&Wine': 0.4090909090909091}

In [13]:
eclat_instance=ECLAT(data=df,verbose=True)
indexes,supports = eclat_instance.fit(min_support=0.4,min_combination=2,max_combination=5,separator='&',verbose=True)
supports

100%|██████████| 6/6 [00:00<00:00, 501.27it/s]


Combination 2 by 2


15it [00:00, 85.69it/s]


Combination 3 by 3


20it [00:00, 106.08it/s]


Combination 4 by 4


15it [00:00, 102.32it/s]


Combination 5 by 5


6it [00:00, 100.27it/s]


{'Apple&Chips': 0.45454545454545453,
 'Apple&Wine': 0.5,
 'Apple&Bread': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Milk': 0.5,
 'Chips&Wine': 0.4090909090909091,
 'Chips&Bread': 0.4090909090909091,
 'Chips&Butter': 0.4090909090909091,
 'Chips&Milk': 0.45454545454545453,
 'Wine&Bread': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Wine&Milk': 0.6363636363636364,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Milk': 0.5909090909090909,
 'Butter&Milk': 0.5909090909090909,
 'Apple&Wine&Bread': 0.45454545454545453,
 'Apple&Wine&Milk': 0.4090909090909091,
 'Apple&Bread&Butter': 0.4090909090909091,
 'Apple&Bread&Milk': 0.4090909090909091,
 'Apple&Butter&Milk': 0.4090909090909091,
 'Wine&Bread&Butter': 0.45454545454545453,
 'Wine&Bread&Milk': 0.5,
 'Wine&Butter&Milk': 0.45454545454545453,
 'Bread&Butter&Milk': 0.5,
 'Wine&Bread&Butter&Milk': 0.4090909090909091}

## Test 2

min_support = 0.2, min_combination = 2, max_combination =4

In [14]:
output = vertical_apriori(records,0.2, min_combination=2, max_combination=4)
res_dict={}
for res in  output:
    res_dict['&'.join(res.items)]=res.support
res_dict

{'Bread&Apple': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Chips': 0.45454545454545453,
 'Milk&Apple': 0.5,
 'Wine&Apple': 0.5,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Chips': 0.4090909090909091,
 'Milk&Bread': 0.5909090909090909,
 'Bread&Wine': 0.5909090909090909,
 'Butter&Chips': 0.4090909090909091,
 'Milk&Butter': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Milk&Chips': 0.45454545454545453,
 'Wine&Chips': 0.4090909090909091,
 'Milk&Wine': 0.6363636363636364,
 'Bread&Apple&Butter': 0.4090909090909091,
 'Bread&Apple&Chips': 0.36363636363636365,
 'Milk&Bread&Apple': 0.4090909090909091,
 'Bread&Apple&Wine': 0.45454545454545453,
 'Apple&Butter&Chips': 0.36363636363636365,
 'Milk&Apple&Butter': 0.4090909090909091,
 'Wine&Apple&Butter': 0.36363636363636365,
 'Milk&Apple&Chips': 0.3181818181818182,
 'Wine&Apple&Chips': 0.2727272727272727,
 'Milk&Apple&Wine': 0.4090909090909091,
 'Bread&Butter&Chips': 0.36363636363636365,
 'Milk&Bread&Butter': 0.5,
 'Bread&Butter&Wine': 0.45454545

In [15]:
eclat_instance=ECLAT(data=df,verbose=True)
indexes,supports = eclat_instance.fit(min_support=0.2,min_combination=2,max_combination=4,separator='&',verbose=True)
supports

100%|██████████| 6/6 [00:00<00:00, 1002.66it/s]


Combination 2 by 2


15it [00:00, 89.52it/s]


Combination 3 by 3


20it [00:00, 71.86it/s]


Combination 4 by 4


15it [00:00, 62.79it/s]


{'Apple&Chips': 0.45454545454545453,
 'Apple&Wine': 0.5,
 'Apple&Bread': 0.5454545454545454,
 'Apple&Butter': 0.5,
 'Apple&Milk': 0.5,
 'Chips&Wine': 0.4090909090909091,
 'Chips&Bread': 0.4090909090909091,
 'Chips&Butter': 0.4090909090909091,
 'Chips&Milk': 0.45454545454545453,
 'Wine&Bread': 0.5909090909090909,
 'Wine&Butter': 0.5,
 'Wine&Milk': 0.6363636363636364,
 'Bread&Butter': 0.5909090909090909,
 'Bread&Milk': 0.5909090909090909,
 'Butter&Milk': 0.5909090909090909,
 'Apple&Chips&Wine': 0.2727272727272727,
 'Apple&Chips&Bread': 0.36363636363636365,
 'Apple&Chips&Butter': 0.36363636363636365,
 'Apple&Chips&Milk': 0.3181818181818182,
 'Apple&Wine&Bread': 0.45454545454545453,
 'Apple&Wine&Butter': 0.36363636363636365,
 'Apple&Wine&Milk': 0.4090909090909091,
 'Apple&Bread&Butter': 0.4090909090909091,
 'Apple&Bread&Milk': 0.4090909090909091,
 'Apple&Butter&Milk': 0.4090909090909091,
 'Chips&Wine&Bread': 0.3181818181818182,
 'Chips&Wine&Butter': 0.2727272727272727,
 'Chips&Wine&Milk': 

## Test 3

min_support = 0.1, min_combination = 3, max_combination = 6

In [16]:
output = vertical_apriori(records,0.1, min_combination=3)
res_dict={}
for res in  output:
    res_dict['&'.join(res.items)]=res.support
res_dict

{'Bread&Apple&Butter': 0.4090909090909091,
 'Bread&Apple&Chips': 0.36363636363636365,
 'Milk&Bread&Apple': 0.4090909090909091,
 'Bread&Apple&Wine': 0.45454545454545453,
 'Apple&Butter&Chips': 0.36363636363636365,
 'Milk&Apple&Butter': 0.4090909090909091,
 'Wine&Apple&Butter': 0.36363636363636365,
 'Milk&Apple&Chips': 0.3181818181818182,
 'Wine&Apple&Chips': 0.2727272727272727,
 'Milk&Apple&Wine': 0.4090909090909091,
 'Bread&Butter&Chips': 0.36363636363636365,
 'Milk&Bread&Butter': 0.5,
 'Bread&Butter&Wine': 0.45454545454545453,
 'Milk&Bread&Chips': 0.3181818181818182,
 'Bread&Wine&Chips': 0.3181818181818182,
 'Milk&Bread&Wine': 0.5,
 'Milk&Butter&Chips': 0.3181818181818182,
 'Wine&Butter&Chips': 0.2727272727272727,
 'Milk&Butter&Wine': 0.45454545454545453,
 'Milk&Wine&Chips': 0.36363636363636365,
 'Bread&Apple&Butter&Chips': 0.3181818181818182,
 'Milk&Bread&Apple&Butter': 0.3181818181818182,
 'Bread&Apple&Butter&Wine': 0.3181818181818182,
 'Milk&Bread&Apple&Chips': 0.2727272727272727,


In [17]:
eclat_instance=ECLAT(data=df,verbose=True)
indexes,supports = eclat_instance.fit(min_support=0.1,min_combination=3,max_combination=6,separator='&',verbose=True)
supports

100%|██████████| 6/6 [00:00<00:00, 1195.07it/s]


Combination 3 by 3


20it [00:00, 81.84it/s]


Combination 4 by 4


15it [00:00, 69.14it/s]


Combination 5 by 5


6it [00:00, 52.43it/s]


Combination 6 by 6


1it [00:00, 43.59it/s]


{'Apple&Chips&Wine': 0.2727272727272727,
 'Apple&Chips&Bread': 0.36363636363636365,
 'Apple&Chips&Butter': 0.36363636363636365,
 'Apple&Chips&Milk': 0.3181818181818182,
 'Apple&Wine&Bread': 0.45454545454545453,
 'Apple&Wine&Butter': 0.36363636363636365,
 'Apple&Wine&Milk': 0.4090909090909091,
 'Apple&Bread&Butter': 0.4090909090909091,
 'Apple&Bread&Milk': 0.4090909090909091,
 'Apple&Butter&Milk': 0.4090909090909091,
 'Chips&Wine&Bread': 0.3181818181818182,
 'Chips&Wine&Butter': 0.2727272727272727,
 'Chips&Wine&Milk': 0.36363636363636365,
 'Chips&Bread&Butter': 0.36363636363636365,
 'Chips&Bread&Milk': 0.3181818181818182,
 'Chips&Butter&Milk': 0.3181818181818182,
 'Wine&Bread&Butter': 0.45454545454545453,
 'Wine&Bread&Milk': 0.5,
 'Wine&Butter&Milk': 0.45454545454545453,
 'Bread&Butter&Milk': 0.5,
 'Apple&Chips&Wine&Bread': 0.2727272727272727,
 'Apple&Chips&Wine&Butter': 0.22727272727272727,
 'Apple&Chips&Wine&Milk': 0.22727272727272727,
 'Apple&Chips&Bread&Butter': 0.3181818181818182,
